In [1]:
using ITensors

# Basic Introduction

In [9]:
# Example of building a simple TTN manually
nsites = 4
sites = [Index(2, "site $i") for i in 1:nsites]

4-element Vector{Index{Int64}}:
 (dim=2|id=23|"site1")
 (dim=2|id=157|"site2")
 (dim=2|id=846|"site3")
 (dim=2|id=259|"site4")

In ITensors when we define an index, we are implicitly defining a dimension for a generic vector space. In this case we are definig 4 bidimensional spaces (for example each one can be associated with a spin).

We can use indexes to define and work with tensors. For example defining: 

-- ITensor(index(2,a)) creates a rank-1 tensor of dimension 2, so a vector of lenght 2.

-- ITensor(index(2,a), index*(2,a)) creates a 2x2 matrix, so a transformation between the two 2d spaces


Contraction is a sum over indexes with same name.

In [ ]:
# Leaf tensors
leaf1 = randomITensor(sites[1], sites[2])
leaf2 = randomITensor(sites[3], sites[4])

# Parent tensor
parent = randomITensor(sites[1], sites[2], sites[3], sites[4])

# contracting
result = parent * leaf1 * leaf2

println("Result of contracting the TTN with random vectors: ", result)

In [ ]:
ABC = @visualize leaf1 * leaf2 * parent edge_labels=(tags=true,);

Contracting parent, a ITensor(i1,i2,i3,i4) with the two leaves, ITensor(i1,i2) and ITensor(i3,i4) returns a scalar since we are summing over all indexes.

# Tree-TensorNetwork

<img src="./Screenshot from 2025-01-13 10-35-28.png" alt="Example Image" width="500"/>

As illustrated in Fig. 1(b), each circle represents
a tensor; each edge of the circle represents an individual
index of the tensor. The first tensor is a matrix connecting the
second and third tensors, while the remaining tensors are all
three-order tensors with three indices. The index between two
tensors is called a virtual bond, which would be contracted
hereafter. The left and right indices of the tensors in the
bottom of the TTN are respectively connected to two pixels
of the input image and hence are called physical bonds.

The bond dimension in a tensor network refers to the dimension of the shared index (or bond) connecting two tensors. 
Higher bond dimensions allow more complex entanglement between subsystems to be represented. In quantum many-body systems, the bond dimension represents the number of states used to approximate the entanglement structure.

Since bond dimension represents on how many indices we are summing, it is related to how well we can represent states with our tensor network.
In the context of Matrix Product States (MPS), the bond dimension directly corresponds to the Schmidt rank of the state for each bipartition. If the bond dimension is h , the MPS can capture states with a maximum Schmidt rank of h.

$|\psi\rangle=\sum_{i=1}^D \lambda_i\left|u_i\right\rangle_A \otimes\left|v_i\right\rangle_B$, where D is the Schmidt rank, the number of non-zero $\lambda$, which quantifies the enganglement between the two subsystems A and B (for D=1 the state is separable).

In [21]:
# let's make a TTN with 3 layers, consisting of rank-3 tensors with bond dimension 2 with random elements

# Define physical indices
n_physical_indexes = 8
physical_indexes = [Index(2, "site $i") for i in 1:n_physical_indexes]

# Define bond indices for each layer counting from the bottom
n_bond_indexes_layer1 = div(n_physical_indexes, 2) # 8 bond indexes
n_bond_indexes_layer2 = div(n_bond_indexes_layer1, 2)   # 4 bond indexes

# Define bond indices
bond_dimension = 2
bond_indexes_layer1 = [Index(bond_dimension, "bond $i") for i in 1:n_bond_indexes_layer1]
bond_indexes_layer2 = [Index(bond_dimension, "bond $i") for i in 1:n_bond_indexes_layer2]

# Define tensors and normalize them
layer1_tensors = [ randomITensor(physical_indexes[2*i-1], physical_indexes[2*i], bond_indexes_layer1[i]) for i in 1:n_bond_indexes_layer1 ]
layer1_tensors = [ normalize!(t) for t in layer1_tensors ]
layer2_tensors = [ randomITensor(bond_indexes_layer1[2*i-1], bond_indexes_layer1[2*i], bond_indexes_layer2[i]) for i in 1:n_bond_indexes_layer2 ]
layer2_tensors = [ normalize!(t) for t in layer2_tensors ]
root_tensor = randomITensor(bond_indexes_layer2[1], bond_indexes_layer2[2])
root_tensor = normalize!(root_tensor)

ITensor ord=2 (dim=2|id=816|"bond1") (dim=2|id=895|"bond2")
NDTensors.Dense{Float64, Vector{Float64}}

In [36]:
# Contracting
result = root_tensor
for t in layer2_tensors
    result = result * t
end
for t in layer1_tensors
    result = result * t
end
println("Result of contracting the TTN: ", result)

Result of contracting the TTN: ITensor ord=8
Dim 1: (dim=2|id=264|"site1")
Dim 2: (dim=2|id=14|"site2")
Dim 3: (dim=2|id=424|"site3")
Dim 4: (dim=2|id=662|"site4")
Dim 5: (dim=2|id=946|"site5")
Dim 6: (dim=2|id=262|"site6")
Dim 7: (dim=2|id=25|"site7")
Dim 8: (dim=2|id=156|"site8")
NDTensors.Diag{Float64, Float64}
 2×2×2×2×2×2×2×2
[:, :, 1, 1, 1, 1, 1, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2, 1, 1, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 1, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 2, 1, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 2, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 1, 2, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 2, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 2, 2, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 1, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 1, 1, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 1, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 2, 1, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 2, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 1, 2, 2, 1, 1] =
 0.0  0.0
 0.0  

### Canonicalization

To simplify things, we need to bring our tensors in canonical form, in which each tensor is in canonical form for an index (in following case for index $\alpha_2$)
$$
\sum_{\alpha_4, \alpha_5} T_{\alpha_2, \alpha_4, \alpha_5}^{[2]} T_{\alpha_2^{\prime}, \alpha_4, \alpha_5}^{[2]}=\delta_{\alpha_2, \alpha_2^{\prime}},
$$

We can do so, by using QR decomposition. Starting from the leaves, we can decompose each tensor in a Unitary part and a Residual part. Then propagating the residual part towards the root we can for a "central tensor" containing all non-canonical information.

In [29]:
# QR decomposition of a tensor as example
# Example of building a simple TTN manually
nsites = 3
sites = [Index(2, "site $i") for i in 1:nsites]

A = randomITensor(sites[1], sites[2], sites[3])
A = A / norm(A)
Q, R = qr(A, [sites[1], sites[2]], [sites[3]])
println("norma di Q: ", norm(Q))
println("norma di R: ", norm(R))

norma di Q: 1.414213562373095
norma di R: 0.9999999999999999


In [15]:
A_reconstructed = Q * R
println("Reconstructed tensor: ", A_reconstructed)
println("Reconstruction error: ", norm(A - A_reconstructed))


Reconstructed tensor: ITensor ord=3
Dim 1: (dim=2|id=969|"site1")
Dim 2: (dim=2|id=200|"site2")
Dim 3: (dim=2|id=208|"site3")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
  0.048089857450470674   0.7699331237566488
 -0.524060519759655     -0.19138062895103594

[:, :, 2] =
 0.1785123678816708   -0.10475232562243073
 0.19817155433257738   0.10729723616035085
Reconstruction error: 1.5639472591400866e-16


In [17]:
# Check that Q is unitary
# Normalize Q
Q = Q
Q_dagger = dag(Q)
result = Q * Q_dagger
println("Q * Q_dagger: ", result)

Q * Q_dagger: ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
1.0


### Canonicalization of TTN

In [1]:
A = layer1_tensors[1]
Q, R = qr(A, [physical_indexes[1], physical_indexes[2]], [bond_indexes_layer1[1]])

UndefVarError: UndefVarError: `layer1_tensors` not defined

In [8]:
layer1_tensors[3]

ITensor ord=3 (dim=2|id=624|"site5") (dim=2|id=745|"site6") (dim=2|id=436|"Link,qr")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
# Canonicalization algorithm:

# osservazione: tutti i nodi dell'albero sono normalizzati, sto normalizzando tutti i Q e spostando la norma su R
# ricordarsi di rieseguire la cella di definizione albero prima di questa

println("Starting canonical conversion...")
println("Layer 1")
for i in 1:length(layer1_tensors)
    A = layer1_tensors[i]
    Q, R = qr(A, [physical_indexes[2*i-1], physical_indexes[2*i]], [bond_indexes_layer1[i]])
    Q = Q / norm(Q)
    R = R * norm(Q)
    layer1_tensors[i] = Q
    layer2_tensors[Int(ceil(i/2))] = layer2_tensors[Int(ceil(i/2))] * R
    println("decomposed tensor $i in layer 1")
end

println("Layer 2")
for i in 1:length(layer2_tensors)
    A = layer2_tensors[i]
    Q, R = qr(A, [bond_indexes_layer1[2*i-1], bond_indexes_layer1[2*i]], [bond_indexes_layer2[i]]) #errore qua
    Q = Q / norm(Q)
    R = R * norm(Q)
    layer2_tensors[i] = Q
    root_tensor = root_tensor * R
    println("decomposed tensor $i in layer 2")
end

println("Root tensor assumed as central tensor")

Starting canonical conversion...
Layer 1
decomposed tensor 1 in layer 1
decomposed tensor 2 in layer 1
decomposed tensor 3 in layer 1
decomposed tensor 4 in layer 1
Layer 2
ok


ErrorException: In `permute(::ITensor, inds...)`, the input ITensor has indices: 

((dim=2|id=555|"CMB,Link"), (dim=2|id=219|"Link,qr"), (dim=2|id=70|"Link,qr"), (dim=1|id=730|"CMB,Link"))

but the desired Index ordering is: 

((dim=1|id=730|"CMB,Link"), (dim=2|id=555|"CMB,Link"))